In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Environment
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import unary_union
import numpy as np
import folium
import hashlib

ml_client = MLClient.from_config(credential=DefaultAzureCredential(), path="config.json")

Found the config file in: config.json


In [2]:
data_asset = ml_client.data.get(name="lisboa_feb_march", version="1")

# Get the actual path (usually a URI to blob or datalake)
data_path = data_asset.path

In [13]:
df = pd.read_parquet(data_path)

In [5]:
df.columns

Index(['unique_id', 'hour_band_id', 'a_imsi', 'wrgc_a_msisdn', 'call_id',
       'cal_year_id', 'cal_month_id', 'cal_day_id', 'a_bts_cgi', 'a_lac_id',
       'a_country', 'wrgo_a_msisdn', 'time_id', 'cgi', 'cgi_key', 'ci',
       'lac_tac', 'nodeb_id', 'localcellid_4g', 'sector_name', 'ref_code',
       'live_sran_sitename', 'technology', 'latitude_cell', 'longitude_cell',
       'bts_id', 'siteid_localcellid', 'cp7', 'freguesia', 'concelho', 'city',
       'ci_sac_eci_nci', 'sac', 'eci', 'active_state', 'cell_mobility',
       'cell_type', 'indoor_flag', 'ambiente', 'asset_cell_status', 'band',
       'ref_tec_name', 'node_id', 'repeated_cellname', 'cellradius_4g',
       'cellradius_5g', 'azimuth', 'added LAC', 'r', 'azi_min1', 'azi_max1',
       'event_date', 'count', 'distrito'],
      dtype='object')

In [6]:
df['cgi']

0           268-03-2510-26642
1           268-03-27-2886070
2           268-03-28-2972814
3           268-03-15-2912950
4            268-03-2-2934966
                  ...        
43748229    268-03-15-2870967
43748230    268-03-27-2924461
43748231     268-03-1-2818731
43748232     268-03-2-2858689
43748233    268-03-15-2897077
Name: cgi, Length: 43748234, dtype: object

In [22]:
network_asset = ml_client.data.get("network_file_v7", version="1")
network = pd.read_csv(network_asset.path)

/tmp/ipykernel_20146/213870635.py:2: DtypeWarning: Columns (0,6,7,8,9,14,15,16,17,18,22,23,24,25,26,27,28,29,35) have mixed types. Specify dtype option on import or set low_memory=False.
  network = pd.read_csv(network_asset.path)


In [24]:
df.dtypes

unique_id                     object
hour_band_id                  object
a_imsi                        object
wrgc_a_msisdn                 object
call_id                       object
cal_year_id                   object
cal_month_id                  object
cal_day_id                     int32
a_bts_cgi             string[python]
a_lac_id                      object
a_country                     object
wrgo_a_msisdn                 object
time_id                       object
cgi                           object
cgi_key                       object
ci                            object
lac_tac                       object
nodeb_id                      object
localcellid_4g                object
sector_name                   object
ref_code                      object
live_sran_sitename            object
technology                    object
latitude_cell                float32
longitude_cell               float32
bts_id                        object
siteid_localcellid            object
c

In [ ]:
#network['cgi_key'] = network['cgi_key'].astype("string").str.strip().astype("int")
network['cell_id'] = network['cgi']
df['a_bts_cgi'] = df['a_bts_cgi'].astype("int")

In [30]:
df.columns

Index(['unique_id', 'hour_band_id', 'a_imsi', 'wrgc_a_msisdn', 'call_id',
       'cal_year_id', 'cal_month_id', 'cal_day_id', 'a_bts_cgi', 'a_lac_id',
       'a_country', 'wrgo_a_msisdn', 'time_id', 'cgi', 'cgi_key', 'ci',
       'lac_tac', 'nodeb_id', 'localcellid_4g', 'sector_name', 'ref_code',
       'live_sran_sitename', 'technology', 'latitude_cell', 'longitude_cell',
       'bts_id', 'siteid_localcellid', 'cp7', 'freguesia', 'concelho', 'city',
       'ci_sac_eci_nci', 'sac', 'eci', 'active_state', 'cell_mobility',
       'cell_type', 'indoor_flag', 'ambiente', 'asset_cell_status', 'band',
       'ref_tec_name', 'node_id', 'repeated_cellname', 'cellradius_4g',
       'cellradius_5g', 'azimuth', 'added LAC', 'r', 'azi_min1', 'azi_max1',
       'event_date', 'count', 'distrito'],
      dtype='object')

In [32]:
df[['unique_id', 'time_id','event_date', 'a_bts_cgi']]

,unique_id,time_id,event_date,a_bts_cgi
0,08GcaR5hWj9xkKRuUeE07UHAv+zEUjOq/i1VZWPtcPPM=_...,2024-02-09 08:58:26.0,2024-02-09,2664202510
1,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-02-09 11:56:21.0,2024-02-09,288607000027
2,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-02-27 20:13:20.0,2024-02-27,297281400028
3,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-02-10 18:15:28.0,2024-02-10,291295000015
4,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-02-05 16:29:36.0,2024-02-05,293496600002
...,...,...,...,...
43748229,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-03-07 08:23:50,2024-03-07,287096700015
43748230,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-03-09 14:24:57,2024-03-09,292446100027
43748231,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-03-07 11:58:33,2024-03-07,281873100001
43748232,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-03-05 17:47:06,2024-03-05,285868900002


In [18]:
network

,cgi,cgi_key,ci,lac_tac,nodeb_id,localcellid_4g,sector_name,ref_code,live_sran_sitename,technology,...,node_id,repeated_cellname,cellradius_4g,cellradius_5g,azimuth,added LAC,r,azi_min1,azi_max1,cell_id
0,268-03-16500-1,116500.0,1.0,16500.0,NaN,NaN,MARCO_CURVACEIRA_MPT266N1,MPT266B1,MARCO_CURVACEIRA_MPT266B1,GSM,...,1230.0,NaN,NaN,NaN,0.0,NaN,1156.0,-60.0,95.0,268-03-16500-1
1,268-03-16500-2,216500.0,2.0,16500.0,NaN,NaN,MARCO_CURVACEIRA_MPT266N2,MPT266B1,MARCO_CURVACEIRA_MPT266B1,GSM,...,1230.0,NaN,NaN,NaN,190.0,NaN,1156.0,95.0,215.0,268-03-16500-2
2,268-03-16500-3,316500.0,3.0,16500.0,NaN,NaN,MARCO_CURVACEIRA_MPT266N3,MPT266B1,MARCO_CURVACEIRA_MPT266B1,GSM,...,1230.0,NaN,NaN,NaN,240.0,NaN,1156.0,215.0,300.0,268-03-16500-3
3,268-03-2010-11,1102010.0,11.0,2010.0,NaN,NaN,MIN_ADMIN_INTERNA_TAGUSPARK_DLB154N1,DLB154B1,MIN_ADMIN_INTERNA_TAGUSPARK_DLB154B1,GSM,...,1.0,NaN,NaN,NaN,0.0,NaN,301.0,0.0,0.0,268-03-2010-11
4,NaN,1103800.0,40011.0,3800.0,NaN,161.0,_,NaN,2G_3G-77405_STP VTP001 LC_VTP001B1,UMTS,...,9999.0,NaN,NaN,NaN,0.0,NaN,NaN,270.0,90.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124047,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124048,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124049,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124050,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
joined_df = pd.merge(df, network[['latitude_cell', 'longitude_cell', 'cgi_key', 'cell_id','r', 'azi_min1', 'azi_max1', 'concelho']], left_on='a_bts_cgi', right_on='cgi_key', how='inner')

In [27]:
joined_df

,unique_id,hour_band_id,a_imsi,wrgc_a_msisdn,call_id,cal_year_id,cal_month_id,cal_day_id,a_bts_cgi,a_lac_id,...,azi_min1_x,azi_max1_x,event_date,count,distrito,cgi_key_y,cell_id,r_y,azi_min1_y,azi_max1_y
0,08GcaR5hWj9xkKRuUeE07UHAv+zEUjOq/i1VZWPtcPPM=_...,17.0,08GcaR5hWj9xkKRuUeE07UHAv+zEUjOq/i1VZWPtcPPM=,35196795XXXX,194872947,2024,202402,20240209,2664202510,2510,...,75,200,2024-02-09,168,LISBOA,2.664203e+09,268-03-2510-26642,154.0,75.0,200.0
1,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,23.0,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=,35191751XXXX,292474939,2024,202402,20240209,288607000027,27,...,30,150,2024-02-09,294,LISBOA,2.886070e+11,268-03-27-2886070,127.0,30.0,150.0
2,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,40.0,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=,35193400XXXX,2097145568,2024,202402,20240227,297281400028,28,...,0,0,2024-02-27,125,LISBOA,2.972814e+11,268-03-28-2972814,64.0,0.0,0.0
3,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,36.0,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=,35196156XXXX,693344727,2024,202402,20240210,291295000015,15,...,120,240,2024-02-10,184,LISBOA,2.912950e+11,268-03-15-2912950,158.0,120.0,240.0
4,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,32.0,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=,35191759XXXX,-1206945904,2024,202402,20240205,293496600002,2,...,90,270,2024-02-05,191,LISBOA,2.934966e+11,268-03-2-2934966,241.0,90.0,270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43748229,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,16,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=,35193589XXXX,1001311850,2024,202403,20240307,287096700015,15,...,-100,185,2024-03-07,2592,LISBOA,2.870967e+11,268-03-15-2870967,210.0,-100.0,185.0
43748230,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,28,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=,35193341XXXX,1900742856,2024,202403,20240309,292446100027,27,...,290,410,2024-03-09,157,LISBOA,2.924461e+11,268-03-27-2924461,47.0,290.0,410.0
43748231,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,23,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=,35196178XXXX,1104196849,2024,202403,20240307,281873100001,1,...,340,100,2024-03-07,163,LISBOA,2.818731e+11,268-03-1-2818731,314.0,340.0,100.0
43748232,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,35,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=,35191725XXXX,300455373,2024,202403,20240305,285868900002,2,...,280,400,2024-03-05,105,LISBOA,2.858689e+11,268-03-2-2858689,139.0,280.0,400.0


In [9]:
network

,cgi,cgi_key,ci,lac_tac,nodeb_id,localcellid_4g,sector_name,ref_code,live_sran_sitename,technology,...,ref_tec_name,node_id,repeated_cellname,cellradius_4g,cellradius_5g,azimuth,added LAC,r,azi_min1,azi_max1
0,268-03-16500-1,116500.0,1.0,16500.0,NaN,NaN,MARCO_CURVACEIRA_MPT266N1,MPT266B1,MARCO_CURVACEIRA_MPT266B1,GSM,...,MARCO_CURVACEIRA_GSM_MPT266B1,1230.0,NaN,NaN,NaN,0.0,NaN,1156.0,-60.0,95.0
1,268-03-16500-2,216500.0,2.0,16500.0,NaN,NaN,MARCO_CURVACEIRA_MPT266N2,MPT266B1,MARCO_CURVACEIRA_MPT266B1,GSM,...,MARCO_CURVACEIRA_GSM_MPT266B1,1230.0,NaN,NaN,NaN,190.0,NaN,1156.0,95.0,215.0
2,268-03-16500-3,316500.0,3.0,16500.0,NaN,NaN,MARCO_CURVACEIRA_MPT266N3,MPT266B1,MARCO_CURVACEIRA_MPT266B1,GSM,...,MARCO_CURVACEIRA_GSM_MPT266B1,1230.0,NaN,NaN,NaN,240.0,NaN,1156.0,215.0,300.0
3,268-03-2010-11,1102010.0,11.0,2010.0,NaN,NaN,MIN_ADMIN_INTERNA_TAGUSPARK_DLB154N1,DLB154B1,MIN_ADMIN_INTERNA_TAGUSPARK_DLB154B1,GSM,...,MIN_ADMIN_INTERNA_TAGUSPARK_GSM_DLB154B1,1.0,NaN,NaN,NaN,0.0,NaN,301.0,0.0,0.0
4,NaN,1103800.0,40011.0,3800.0,NaN,161.0,_,NaN,2G_3G-77405_STP VTP001 LC_VTP001B1,UMTS,...,_UMTS_,9999.0,NaN,NaN,NaN,0.0,NaN,NaN,270.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rivers_gdf = gpd.read_file("hotosm_prt_waterways_polygons_geojson.geojson")

In [6]:
rivers_gdf = rivers_gdf.loc[(rivers_gdf['name:en'] == 'Tagus River') & (rivers_gdf['osm_type'] == 'ways_poly')]

In [7]:
def generate_cell_id(row):
    # Create a unique string combining tower location and azimuth range
    unique_str = f"{row['ref_code']}_{row['azimuth']}"
    # Use a hash to make a numeric ID
    return int(hashlib.sha1(unique_str.encode()).hexdigest(), 16) % (10**10)  # 10-digit number

df["cell_id"] = network.apply(generate_cell_id, axis=1)


In [ ]:
# ---------- UTILS ----------
def telco_to_math_angle(telco_angle):
    """Convert telco azimuth (0°=North, CW) to math angle (0°=East, CCW)."""
    return (90 - telco_angle) % 360

def adjust_azimuth_for_omni(azimuth_min_telco, azimuth_max_telco):
    """Convert azimuths and handle omni-directional case."""
    try:
        azimuth_min_telco = float(azimuth_min_telco) if not pd.isna(azimuth_min_telco) else 0
    except ValueError:
        azimuth_min_telco = 0

    try:
        azimuth_max_telco = float(azimuth_max_telco) if not pd.isna(azimuth_max_telco) else 0
    except ValueError:
        azimuth_max_telco = 0

    # Handle omni-directional case
    if azimuth_min_telco == 0 and azimuth_max_telco == 0:
        return 0, 360  
    
    az_min = telco_to_math_angle(azimuth_min_telco)
    az_max = telco_to_math_angle(azimuth_max_telco)
    
    if az_max < az_min:
        az_max += 360
    return az_min, az_max

def make_sector_projected(x, y, az_min, az_max, radius_m, num_points=60):
    """Create sector polygon from projected coords."""
    angles = np.linspace(np.deg2rad(az_min), np.deg2rad(az_max), num_points)
    arc_points = [(x + radius_m * np.cos(a), y + radius_m * np.sin(a)) for a in angles]
    points = gpd.GeoSeries(
    [Point(x, y)] + [Point(px, py) for px, py in arc_points])
    sector = points.union_all().convex_hull
    return sector


def deterministic_point_in_polygon(user_id, cell_id, polygon):
    """Pick stable location per (user, cell) pair."""
    if polygon.is_empty:
        return None
    minx, miny, maxx, maxy = polygon.bounds
    rng = np.random.default_rng(abs(hash((user_id, cell_id))) % (2**32))
    for _ in range(50):  # up to 50 tries to find point inside polygon
        px = rng.uniform(minx, maxx)
        py = rng.uniform(miny, maxy)
        point = Point(px, py)
        if polygon.contains(point):
            return point
    return polygon.centroid

# ---------- MAIN PROCESSOR ----------
def process_cdr_locations_vectorized(df, rivers_gdf, radius_km=1.0, crs_proj="EPSG:3763"):
    """Fully vectorized approach for CDR processing."""
    
    # Prepare rivers projection once
    rivers_proj = rivers_gdf.to_crs(crs_proj)
    water_union = rivers_proj.union_all()  # Shapely >= 2.0
    
    towers_df = df[["cell_id", "longitude_cell", "latitude_cell"]].drop_duplicates()

    towers_gdf = gpd.GeoDataFrame(
        towers_df,
        geometry=gpd.points_from_xy(towers_df["longitude_cell"], towers_df["latitude_cell"]),
        crs="EPSG:4326"
    )

    # Compute sectors for all towers at once
    towers_gdf["az_min"] = towers_gdf["cell_id"].map(
        df.groupby("cell_id")["azi_min1"].first()
    )
    towers_gdf["az_max"] = towers_gdf["cell_id"].map(
        df.groupby("cell_id")["azi_max1"].first()
    )

    # Build all sectors in one go
    towers_gdf["sector_poly"] = [
        make_sector_projected(pt.x, pt.y, *adjust_azimuth_for_omni(az_min, az_max), radius_km * 1000)
        for pt, az_min, az_max in zip(towers_gdf.geometry, towers_gdf["az_min"], towers_gdf["az_max"])
    ]

    # Remove water areas in one vectorized difference
    sectors_gdf = gpd.GeoDataFrame(towers_gdf[["cell_id"]], geometry=towers_gdf["sector_poly"], crs=crs_proj)
    sectors_gdf["geometry"] = sectors_gdf.geometry.apply(lambda g: g.difference(water_union))

    # Pick largest polygon for multipolygons (vectorized)
    sectors_gdf["geometry"] = sectors_gdf.geometry.apply(
        lambda g: max(g.geoms, key=lambda gg: gg.area) if g.geom_type == "MultiPolygon" else g
    )

    # Step 1 — Ensure one row per tower in sectors_gdf
    sectors_gdf = sectors_gdf.drop_duplicates(subset=["cell_id"])

    # Step 2 — Merge safely
    df = df.merge(
        sectors_gdf[["cell_id", "geometry"]],
        on="cell_id",
        how="left",
        validate="many_to_one"  # <- catches many-to-many mistakes
)


    # Generate deterministic points for all users in one apply
    df["point_proj"] = [
        deterministic_point_in_polygon(uid, cid, poly) if poly and not poly.is_empty else None
        for uid, cid, poly in zip(df["unique_id"], df["cell_id"], df["geometry"])
    ]

    # Convert all points to WGS84 at once
    points_gdf = gpd.GeoDataFrame(df, geometry=df["point_proj"], crs=crs_proj).to_crs("EPSG:4326")
    df["est_lon"] = points_gdf.geometry.x
    df["est_lat"] = points_gdf.geometry.y

    return df#.drop(columns="geometry")


In [16]:
df_result = process_cdr_locations_vectorized(
    df,               # Must have user_id, cell_id
    rivers_gdf
)


In [19]:
df_result[['longitude_cell', 'latitude_cell', 'est_lon', 'est_lat']]

,longitude_cell,latitude_cell,est_lon,est_lat
0,-9.038759,38.897488,-8.136140,39.676949
1,-9.230803,38.702778,-8.136811,39.669715
2,-9.147932,38.707329,-8.128866,39.668236
3,-9.263709,38.759209,-8.130347,39.671758
4,-9.112000,38.757893,-8.143481,39.671837
...,...,...,...,...
43748229,-9.268195,38.757820,-8.143626,39.665944
43748230,-9.238561,38.725662,-8.134948,39.666690
43748231,-9.107435,38.814362,-8.140413,39.662607
43748232,-9.138016,38.742714,-8.140035,39.668171


In [20]:
df_result.to_csv("/home/azureuser/cloudfiles/code/Users/20230692/output_files/cdr_location_spreading.csv", index=False)

KeyboardInterrupt: 

In [24]:
df_ = df_result.drop(columns="geometry")

In [25]:
df_.columns

Index(['unique_id', 'hour_band_id', 'a_imsi', 'wrgc_a_msisdn', 'call_id',
       'cal_year_id', 'cal_month_id', 'cal_day_id', 'a_bts_cgi', 'a_lac_id',
       'a_country', 'wrgo_a_msisdn', 'time_id', 'cgi', 'cgi_key', 'ci',
       'lac_tac', 'nodeb_id', 'localcellid_4g', 'sector_name', 'ref_code',
       'live_sran_sitename', 'technology', 'latitude_cell', 'longitude_cell',
       'bts_id', 'siteid_localcellid', 'cp7', 'freguesia', 'concelho', 'city',
       'ci_sac_eci_nci', 'sac', 'eci', 'active_state', 'cell_mobility',
       'cell_type', 'indoor_flag', 'ambiente', 'asset_cell_status', 'band',
       'ref_tec_name', 'node_id', 'repeated_cellname', 'cellradius_4g',
       'cellradius_5g', 'azimuth', 'added LAC', 'r', 'azi_min1', 'azi_max1',
       'event_date', 'count', 'distrito', 'cell_id', 'point_proj', 'est_lon',
       'est_lat'],
      dtype='object')

In [27]:
df_ = df_[['unique_id', 'time_id', 'technology', 'latitude_cell', 'longitude_cell','concelho', 'city','indoor_flag', 'r', 'azi_min1', 'azi_max1',
       'event_date', 'distrito', 'cell_id', 'point_proj', 'est_lon',
       'est_lat']]

In [28]:
df_.to_pickle("/home/azureuser/cloudfiles/code/Users/20230692/output_files/cdr_location_spreading_result.pkl")  

In [29]:
import branca.colormap as cm
from sklearn.preprocessing import MinMaxScaler

def visualize_user_locations_lisbon_scaled(df, output_html="user_locations_map_lisbon.html"):
    """
    Visualizes estimated user locations and number of users per location on a Folium map,
    centered on Lisbon, with proper scaling for circle size.
    """
    # Drop NaNs in coordinates
    #df = df.dropna(subset=["est_lon", "est_lat"])

    # Group by location and count users
    grouped = df.groupby(["est_lat", "est_lon"]).size().reset_index(name="user_count")

    # Scale circle radius between 3 and 20 pixels
    scaler = MinMaxScaler(feature_range=(3, 20))
    grouped["radius"] = scaler.fit_transform(grouped[["user_count"]])

    # Define Lisbon center coordinates
    lisbon_center = [38.7169, -9.139]  # Lisbon approx

    # Create base map centered on Lisbon
    m = folium.Map(location=lisbon_center, zoom_start=11, tiles="OpenStreetMap")

    # Create colormap for user counts
    colormap = cm.linear.YlOrRd_09.scale(grouped["user_count"].min(), grouped["user_count"].max())
    colormap.caption = "Number of Users"
    colormap.add_to(m)

    # Add points to map
    for _, row in grouped.iterrows():
        folium.CircleMarker(
            location=[row["est_lat"], row["est_lon"]],
            radius=row["radius"],  # scaled radius
            color=colormap(row["user_count"]),
            fill=True,
            fill_opacity=0.6,
            popup=f"Users: {row['user_count']}"
        ).add_to(m)

    # Save to HTML
    m.save(output_html)
    print(f"Map saved to {output_html}")

# Example usage:
visualize_user_locations_lisbon_scaled(df_, "user_locations_map_lisbon.html")

KeyboardInterrupt: 